In [ ]:
from flask import Flask, redirect, url_for, session
from flask_oidc import OpenIDConnect
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Import Flask server from keycloak-auth.py
from keycloakAuth import flaskServer, oidc

# Initialize Dash app with Flask server
app = dash.Dash(__name__, server=flaskServer, url_base_pathname='/dashboard/')

# Layout of the dashboard
app.layout = html.Div(id="main", children=[
    html.H1('Secure Dashboard'),
    html.Div(id='connection-status'),
    html.Div(id='content'), 
    html.Button(id='login-logout-button', n_clicks=0),
    html.Div(id='login-logout')
])

# Dash callback to handle login button click
@app.callback(
    Output('login-logout-button', 'children'),
    [Input('main', 'children')]
)
def login_logout_button(main):
    if oidc.user_loggedin:
        return "Logout"
    else:
        return "Login"
        
# Dash callback to handle login button click
@app.callback(
    Output('login-logout', 'children'),
    [Input('login-logout-button', 'n_clicks')]
)
def login_logout_button_click(n_clicks):
    if n_clicks > 0:
        return dcc.Location(pathname='/', id='login-logout-link')

# Dash callback to update connection status
@app.callback(
    Output('connection-status', 'children'),
    [Input('main', 'children')]
)
def update_connection_status(input_value):
    if oidc.user_loggedin:
        user_info = session.get('oidc_auth_profile', {})
        preferred_username = user_info.get('preferred_username')
        return f'Logged in as {preferred_username}'
    else:
        return 'Not logged in'

# Example of a protected route (not yet implemented)
@app.server.route('/dashboard/')
def render_dashboard():
    return app.index()

# Example of a logout button (not yet implemented)
@app.server.route('/logout')
def logout():
    return redirect(url_for('home'))  # Replace 'home' with your home route

if __name__ == '__main__':
    app.run_server(debug=True, port=5000)
